# Compare local QARTOD Climatology Test Results to Expected Flags
Next, we want to calculate the statistics of the different QARTOD flags for the different tests that are applied to the different parameters in the dataset. The example ```qartod_results_summary``` below simply counts the total number of different flags (e.g 1, 3, 4) and their relative percentages for each test (gross range, climatology, etc) for each parameter that the tests area applied to. 

### Import modules used in this notebook

In [22]:
# Import libraries
import os
import re
import requests
import gc
import io
import ast
import pandas as pd
import numpy as np
import xarray as xr
import warnings
warnings.filterwarnings("ignore")
import sys
import glob

In [23]:
# Import function to build relative path to data files
import qartod_testing.data_processing as dp

### Define reference designator for chosen variable

In [24]:
# Set reference designator, data stream, and method 
method = "recovered_inst"                       
stream = "ctdbp_cdef_instrument_recovered"   
refdes = "CP01CNSM-MFD37-03-CTDBPD000"              

# Site, node, and sensor info from deconstructed reference designator
[site, node, sensor] = refdes.split('-', 2)

### Load local QARTOD test flags from processed dataset

In [25]:
# Build path to data files with local test results and load to workspace - Might not use this
local_test_path = dp.build_data_path(refdes, method, stream, 'clim-result', folder="interim")

# Load local test data from .nc files
local_test_results = xr.open_dataset(local_test_path)

In [26]:
local_test_results

<xarray.Dataset>
Dimensions:                            (time: 7743)
Coordinates:
  * time                               (time) datetime64[ns] 2017-06-09T14:30...
Data variables:
    sea_water_electrical_conductivity  (time) object ...
    sea_water_temperature              (time) uint8 ...
    sea_water_practical_salinity       (time) uint8 ...
    sea_water_pressure                 (time) object ...

### Extract and parse expected QC results

In [27]:
# Load expected results data from external data folder
# build path to folder where data was saved
folder_path = os.path.join(os.path.abspath('../data/external'), method, stream, refdes)

# retrieve list of netCDF files in this directory
files = glob.glob(folder_path+'/*.nc')
file = files[-1]

expected_ds_path = os.path.join(folder_path, file)
ds_expected = xr.open_dataset(expected_ds_path)

# Create a dictionary of key-value pairs of dataset variable name:alternate parameter name
test_parameters={}
for var in ds_expected.variables:
    if "qartod_results" in var:
        # Get the parameter name
        param = var.split("_qartod")[0]
        
        # Check if the parameter has an alternative ooinet_name
        if "alternate_parameter_name" in ds_expected[param].attrs:
            ooinet_name = ds_expected[param].attrs["alternate_parameter_name"]
        else:
            ooinet_name = param
        
        # Save the results in a dictionary
        test_parameters.update({
            param: ooinet_name
        })
# Print out the results
test_parameters

{'sea_water_electrical_conductivity': 'ctdbp_seawater_conductivity',
 'sea_water_temperature': 'ctdbp_seawater_temperature',
 'sea_water_practical_salinity': 'practical_salinity',
 'sea_water_pressure': 'ctdbp_seawater_pressure'}

In [28]:
def parse_qartod_executed(ds, parameters):
    """
    Parses the qartod tests for the given parameter into separate variables.
    
    Parameters
    ----------
    ds: xarray.DataSet
        The dataset downloaded from OOI with the QARTOD flags applied.
    parameters: list[str]
        The name of the parameters in the dataset to parse the QARTOD flags
        
    Returns
    -------
    ds: xarray.DataSet
        The dataset with the QARTOD test for the given parameters split out
        into new seperate data variables using the naming convention:
        {parameter}_qartod_{test_name}
    """
    # For the params into a list if only a string
    if type(parameters) is not list:
        parameters = list(parameters)
    
    # Iterate through each parameter
    for param in parameters:
        # Generate the qartod executed name
        qartod_name = f"{param}_qartod_executed"
        
        if qartod_name not in ds.variables:
            continue
    
        # Fix the test types
        ds[qartod_name] = ds[qartod_name].astype(str)
    
        # Get the test order
        test_order = ds[qartod_name].attrs["tests_executed"].split(",")
    
        # Iterate through the available tests and create separate variables with the results
        for test in test_order:
            test_index = test_order.index(test)
            test_name = f"{param}_qartod_{test.strip()}"
            ds[test_name] = ds[qartod_name].str.get(test_index)

    return ds

In [29]:
# Put the test parameter names in the dataset into a list
parameters = [x for x in test_parameters.keys()]

In [30]:
# Parse all of the variables with QARTOD tests applied into separate tests
results_expected = parse_qartod_executed(ds_expected, parameters)
results_expected

<xarray.Dataset>
Dimensions:                                           (time: 7743)
Coordinates:
  * time                                              (time) datetime64[ns] 2...
Data variables: (12/31)
    sea_water_pressure_qc_results                     (time) uint8 ...
    sea_water_pressure                                (time) float64 ...
    sea_water_electrical_conductivity_qartod_results  (time) uint8 ...
    sea_water_pressure_qc_executed                    (time) uint8 ...
    ctd_time                                          (time) datetime64[ns] ...
    sea_water_practical_salinity_qc_executed          (time) uint8 ...
    ...                                                ...
    conductivity_qc_executed                          (time) uint8 ...
    sea_water_temperature_qc_executed                 (time) uint8 ...
    sea_water_density                                 (time) float64 ...
    depth                                             (time) float64 ...
    sea_water_pressure_qartod_results                 (time) uint8 ...
    sea_water_temperature                             (time) float64 ...
Attributes: (12/69)
    node:                               MFD37
    comment:                            Data collected from the OOI M2M API a...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_inst
    stream:                             ctdbp_cdef_instrument_recovered
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                40.13981667
    lon:                                -70.77115

In [ ]:
results_expected['sea_water_pressure_qartod_results']

### Comparing local results of QARTOD tests to expected results 

In [21]:
# using Andrew's example:

def run_comparison(ds, param, test_results):
    """
    Runs a comparison between the qartod climatology results returned as part of the dataset
    and results calculated locally.
    """
    test_name = f"{param}_qartod_climatology_test"
    if test_name not in ds.variables:
        return None
    
    # Get the local test results and convert to string type for comparison
    local_results = test_results[param].astype(str)
    
    # Run comparison
    not_equal = np.where(ds[f"{param}_qartod_climatology_test"] != local_results)[0]
    
    if len(not_equal) == 0:
        return None
    else:
        return not_equal

In [13]:
# Identify differences in the results
test_comparison = dict()

for index, param in enumerate(parameters):
    print("Checking for mismatched QARTOD flags in "f"{param}")
    flag_mismatch = run_comparison(results_expected, param, local_test_results)

    if flag_mismatch is None:
        print("No mismatched values found")
        pass
    else:  
        flag_mismatch = flag_mismatch[np.char.isnumeric(results_expected[f"{param}_qartod_climatology_test"][flag_mismatch])] 

        if len(flag_mismatch) == 0:
            print("No mismatched values found")
            pass
        else:
           test_comparison.update({f"{param}_mismatched_flags":{
                    "time": results_expected['time'][flag_mismatch].values,
                    "expected flags": results_expected[f"{param}_qartod_climatology_test"][flag_mismatch].values,
                    "local test flags": local_test_results[param][flag_mismatch].values
                }
            })

Checking for mismatched QARTOD flags in sea_water_electrical_conductivity
No mismatched values found
Checking for mismatched QARTOD flags in sea_water_pressure
No mismatched values found
Checking for mismatched QARTOD flags in sea_water_temperature
No mismatched values found
Checking for mismatched QARTOD flags in sea_water_practical_salinity
No mismatched values found


In [14]:
test_comparison

{}

Next, we'll manually create datasets to hold the results of the comparison for each parameter

In [15]:
temperature_mismatch = xr.Dataset(data_vars=dict(expected_flags=(["time"], test_comparison['sea_water_temperature_mismatched_flags']['expected flags']),
                                                 local_test_flags=(["time"], test_comparison['sea_water_temperature_mismatched_flags']['local test flags'])
                                                 ),
                                    coords=dict(time=(["time"], test_comparison['sea_water_temperature_mismatched_flags']['time']))
                                )

KeyError: 'sea_water_temperature_mismatched_flags'

In [115]:
temperature_mismatch

<xarray.Dataset>
Dimensions:           (time: 31051)
Coordinates:
  * time              (time) datetime64[ns] 2015-12-04T10:31:50 ... 2022-08-1...
Data variables:
    expected_flags    (time) <U1 '3' '3' '3' '3' '3' '3' ... '1' '1' '1' '1' '1'
    local_test_flags  (time) uint8 1 1 1 1 1 1 1 1 1 1 1 ... 3 3 3 3 3 3 3 3 3 3

In [122]:
import matplotlib.pyplot as plt
temperature_mismatch.plot
plt.show

ModuleNotFoundError: No module named 'matplotlib'

### Prepare CSV with statistics about QARTOD results

In [18]:
def qartod_results_summary(ds, params, test, test_comparison):
    """
    Calculate the statistics for parameter qartod flags.
    
    This function takes in a list of the parameters and
    the associated QARTOD tests to calculate the number
    of each flag and the percent of the flag.
    
    Parameters
    ----------
    ds: xarray.DataSet
        An xarray dataset which contains the data
    params: list[strings]
        A list of the variables/parameters in the given
        dataset that have been tested with QARTOD
    tests: list[strings]
        A list of the QARTOD test names which to parse
        for the given parameters.
        
    Returns
    -------
    results: dict
        A dictionary which contains the number of each
        QARTOD flag and the percent of the total flags
        for each test applied to each parameter in the
        given dataset.
        
        results = {'parameter':
                        {'test_name':
                            {'total data points': int,
                            'good data points': (int, %),
                            'suspect data points': (int, %),
                            'bad data points': (int, %)}
                            },
                        }
    """
    # Check that the inputs are a list
    if type(params) is not list:
        params = [params]
        
    # Initialize the result dictionary and iterate 
    # through the parameters for each test
    results = {}
    for param in params:
        
        # Now create dictionary of results for test
        test_results = {}
        
        # First, check that the test was applied
        test_name = f"{param}_qartod_{test}_test"
        if test_name not in ds.variables:
            continue
            
        # Count the total number of values
        n = ds[test_name].count().compute().values
        
        # First calculate the gross range results
        good = np.where(ds[test_name] == "1")[0]

        # Count the number of suspect/interesting
        suspect = np.where(ds[test_name] == "3")[0]

        # Count the number of fails
        bad = np.where(ds[test_name] == "4'")[0]

        test_results.update({"total": int(n),
            "good": (len(good), np.round(len(good)/n*100, 2)),
            "suspect": (len(suspect), np.round(len(suspect)/n*100, 2)),
            "fail": (len(bad), np.round(len(bad)/n*100, 2))
            }
        )
    
        if f"{param}_mismatched_flags" in test_comparison:
            for label in test_comparison[f"{param}_mismatched_flags"]: test_results.update({label: (len(test_comparison[f"{param}_mismatched_flags"][label]), np.round(len(test_comparison[f"{param}_mismatched_flags"][label])/n*100, 2))})
    
        # Save the test results for each parameter
        results.update({
            param: test_results
        })
    
    return results

In [19]:
qartod_results = qartod_results_summary(results_expected, parameters, "climatology", test_comparison)
qartod_results

{}

In [20]:
qartod_flag_stats = pd.DataFrame.from_dict(qartod_results, orient='index')
qartod_flag_stats

""


In [117]:
# save data frame with statistics to csv
csv_path = dp.build_data_path(refdes, method, stream, 'qartod-clim-stats', folder='processed', suffix='.csv')
qartod_flag_stats.to_csv(csv_path)

In [119]:
# save datasets of comparison results to .nc file
nc_path = dp.build_data_path(refdes, method, stream, 'qartod-clim-comparison', folder='processed', suffix='.nc')
temperature_mismatch.to_netcdf(nc_path, mode='a')